In [ ]:
#Authors:Group 9
    #Beiyang Yu (5353653)
    #Mazen Alqadi (476578)
    #Tianyang Lu (5215277)
    #Xilin Yin (5271649)

In [1]:
import numpy as np
import math
import pandas as pd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import scipy.integrate as spint
%matplotlib inline

In [2]:
doc1 = pd.read_excel('WieringermeerData_Meteo.xlsx')   # Read the measured data                        

In [3]:
Jrf = np.zeros(2757, dtype=float)      
E = np.zeros(2757, dtype=float)
Jrf[0:2757] = doc1.iloc[3451:6208,1]   
E[0:2757] = doc1.iloc[3451:6208,2]
# Initialization of J and E matrix

In [4]:
acl = 0.007     # 0.005~0.01
awb = 0.0008    # 0.0005~0.001
Scl_max = 0.60  # 0.45~0.975
Swb_max = 7.0   # 4.5~7.8
Scl_min = 0     
Swb_min = 0
beta0 = 0.98    # 0~1
Cf = 0.92       # 0~1.4
fred = 1.0
bcl = 5         # 0~80
bwb = 30        # 0~80
def dYdt(t, Y):
    a = math.ceil(t)
    return np.array([Jrf[a-1] - acl * ((Y[0] - Scl_min) / (Scl_max - Scl_min)) ** bcl -  E[a-1] * Cf * fred,
                    (1 - beta0 * (Y[0] - Scl_min) / (Scl_max - Scl_min)) * acl * ((Y[0] - Scl_min) / (Scl_max - Scl_min)) ** bcl - awb * ((Y[1] - Swb_min) / (Swb_max - Swb_min)) ** bwb])
# Definition of the ODE 

In [5]:
# Initialisation of tOut and t_span
tOut = np.linspace(0, 2757, 2757)    # 6209 points are measured but only 2757 are used
nOut = np.shape(tOut)[0]
t_span = [tOut[0], tOut[-1]]  

In [6]:
# Set up the initial value
Y0 = np.array([Scl_max / 1.547, Swb_max / 1.035])
# Solve ODE
res = spint.solve_ivp(dYdt, t_span, Y0, t_eval=tOut, vectorized=True,
                      method='RK45', rtol=1e-5)

<ipython-input-4-9784718e390c>:15: RuntimeWarning: overflow encountered in power
  (1 - beta0 * (Y[0] - Scl_min) / (Scl_max - Scl_min)) * acl * ((Y[0] - Scl_min) / (Scl_max - Scl_min)) ** bcl - awb * ((Y[1] - Swb_min) / (Swb_max - Swb_min)) ** bwb])


In [ ]:
t = res['t']
Y = res['y']
# The solution for ODE

In [ ]:
rODE = Y[0,:]
fODE = Y[1,:]
tOut1 = np.linspace(0, 2757, 2757)

# Plot Cover layer storage and Waste body storage over time
plt.figure()
plt.plot(tOut1, rODE, 'r-', label='Cover layer')
plt.plot(tOut1, fODE  , 'b-', label='Waste body')
plt.grid()
plt.legend(loc='best')
plt.xlabel('Time (day)')
plt.ylabel('Water storage (m)')
# f1.savefig('Cover_layer_and_Waste_body.png')

# Plot Cover layer storage VS Waste body storage 
# plt.figure()
# plt.plot(rODE,fODE , 'b-', label='ODE')
# plt.grid()
# plt.legend(loc='best')
# plt.xlabel('Cover layer storage (m)')
# plt.ylabel('Waste body storage (m)')

In [ ]:
doc2 = pd.read_excel('WieringermeerData_LeachateProduction.xlsx')   # Read the measured data 
Q_dr = doc2.iloc[:,1]


In [ ]:
tOut = np.linspace(0, 2757, 2757)
nOut = np.shape(tOut)[0]
Q_cal = np.zeros(2757)
Q_rate = np.zeros(2757)
E = np.zeros(2757)
Q_rate[0] =  0.
# Calculate the storage rate from the measured data
for i in range (1, nOut-1):
    Q_rate[i] = Q_dr[i] - Q_dr[i-1]
Q_cal = beta0 * ((rODE - Scl_min) / (Scl_max - Scl_min)) * acl * ((rODE - Scl_min) / (Scl_max - Scl_min)) ** bcl + awb * ((fODE - Swb_min) / (Swb_max - Swb_min)) ** bwb
Q_cal = Q_cal * 28355   # Multiply the area
print(Q_cal)

In [ ]:
plt.figure()
plt.plot(tOut, Q_cal, 'r-', label='Calculated')
plt.plot(tOut, Q_rate, 'b-', label='Measured')
plt.grid()
plt.legend(loc='best')
plt.xlabel('Time (day)')
plt.ylabel('Leachate production rate (m^3/day)')
plt.show()
# Plot the calculated and measured leachate production rate over time

In [ ]:
Qcal = np.zeros(2757)
Qcal[0] = Q_cal[0]
for i in range(1,2757):
    Qcal[i] = Qcal[i-1] + Q_cal[i] 
    
print(Qcal)

aList = []
aList.append(Qcal);
# Save the calculated Qcal for variable parameters

In [ ]:
plt.figure()
plt.plot(tOut, aList[0], 'k-', label=' bwb=0')
plt.plot(tOut, aList[1], 'b-', label=' bwb=16')
plt.plot(tOut, aList[2], 'c-', label=' bwb=32')
plt.plot(tOut, aList[3], 'g-', label=' bwb=48')
plt.plot(tOut, aList[4], 'y-', label=' bwb=64')
plt.plot(tOut, aList[5], 'm-', label=' bwb=80')
plt.plot(tOut, Q_dr, 'r-', label='Measured')

#plt.xlim(1400, 2000)
#plt.ylim(60000, 90000)
# Zoom in the graph

plt.grid()
plt.legend(loc='best')
plt.xlabel('Time (day)')
plt.ylabel('Leachate production (m^3)')
plt.savefig("filename.png")
plt.show()

# Plot the results of variable parameters over time